# Optimizing Quick Commerce Logistics in Kochi

This notebook walks through:

- Fetching Kochi geodata (OSM)
- Creating demand zones
- Preparing candidate facility locations
- Running facility location MILP (PuLP)
- Running VRP (OR-Tools)
- Visualizing results (folium maps)


In [ ]:
import os
from scripts.data_fetch import fetch_koch_city_boundary, fetch_road_network, fetch_pois, create_grid_zones, estimate_zone_demands
from scripts.facility_optimization import solve_facility_location
from scripts.vrp_solver import solve_vrp

# Create data/ folder
os.makedirs('../data', exist_ok=True)

# 1) Fetch Kochi polygon (this requires internet)
koch = fetch_koch_city_boundary()
koch.to_file('../data/koch_boundary.geojson', driver='GeoJSON')
print('Saved Kochi boundary')

# 2) Fetch POIs and road network (OSM)
G = fetch_road_network()
ox.save_graphml(G, '../data/koch_graph.graphml')
print('Saved road graph')

pois = fetch_pois()
pois.to_file('../data/koch_pois.geojson', driver='GeoJSON')
print('Saved POIs')

# 3) Make grid zones
zones = create_grid_zones(koch, cell_size=2000)
zones.to_file('../data/koch_zones.geojson', driver='GeoJSON')
print('Saved zones')

# 4) Estimate demands (replace with real population data if available)
zones = estimate_zone_demands(zones, total_orders_per_day=2500)
zones[['est_orders_per_day']].to_csv('../data/zone_demands.csv')
print('Saved zone demands')

# 5) Prepare candidate facilities from POIs (top supermarkets/warehouses)
# For demo, pick the first 6 POIs as candidates (you should filter by tags)
candidates = pois.head(6).copy()
candidates['fixed_cost'] = [1200,1100,1300,900,1000,1400]
candidates['capacity'] = [800,700,900,600,750,1000]
candidates.to_file('../data/candidates.geojson', driver='GeoJSON')
print('Saved candidate facilities')

# 6) Compute approximate transport cost matrix using centroid distances (Haversine)
import numpy as np
from shapely.geometry import Point

def haversine(a,b):
    import math
    lon1, lat1 = a
    lon2, lat2 = b
    R = 6371
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2-lat1)
    dlambda = math.radians(lon2-lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return R*2*math.atan2(math.sqrt(a), math.sqrt(1-a))

fac_coords = [(geom.y, geom.x) for geom in candidates.geometry.centroid]
zone_centroids = [(geom.y, geom.x) for geom in zones.geometry.centroid]

m = len(fac_coords); n = len(zone_centroids)
transport_cost = np.zeros((m,n))
for i in range(m):
    for j in range(n):
        distance_km = haversine((fac_coords[i][1], fac_coords[i][0]), (zone_centroids[j][1], zone_centroids[j][0]))
        transport_cost[i,j] = distance_km * 10  # cost per km (INR) - adjust

# 7) Run facility optimization
import pandas as pd
fac_df = pd.DataFrame({'fixed_cost': candidates['fixed_cost'].values, 'capacity': candidates['capacity'].values})
zone_df = pd.DataFrame({'demand': zones['est_orders_per_day'].astype(int).values})
res = solve_facility_location(fac_df.reset_index(drop=True), zone_df.reset_index(drop=True), transport_cost)
print('Optimization status:', res['status'])
print('Total cost:', res['total_cost'])

# 8) Save outputs
pd.DataFrame(res['flows']).to_csv('../data/flows.csv', index=False)
pd.DataFrame({'open': res['open']}).to_csv('../data/open_facilities.csv', index=False)
print('Saved outputs to data folder')
